# Introduction
- This notebook is meant to help with the Deployment of our Sample Demo application that will be monitored by Application Health Extension

## Prerequisites
- To be able to Deploy a VMSS with Application Health Extension, and the Demo Application, you need the following:

1. Build Application and Package Application  

In [ ]:
!make bundle-linux

2. Install Azure CLI
   - Install the Azure CLI using bash. This will allow you to interact with Azure directly from the command line.

In [ ]:
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

3. Login to Azure:
   - Use the 'az login' command to log in to your Azure account. This will require you to enter your Azure credentials.

In [ ]:
!az login

4.  Choose Subscription:
    - If Needed Select the subscription you want to use with the 'az account set --subscription' command.

In [ ]:
# Add you Azure SubscriptionId
subId=""

!az account set --subscription $subId 

## Deployment Scripts
  - We will use the azure cli to deploy a new linux VMSS with Application Health Extension 2.0 enabled and the sample demo application for Application Health Extension to monitor.

1. Create a New Resource Group:
    - Use the 'az group create' command to create a new resource group. You will need to specify the name and location of the resource group.

In [ ]:
# Define the resource group name and location
resource_group_name="" # Add your Resource Group Name
location="eastus2" # Change the location if needed

# Create a new resource group
!az group create --name $resource_group_name --location $location

2. Create a Storage:

In [ ]:
storage_account_name="" # Add your Storage Account Name
!az storage account create --name $storage_account_name --resource-group $resource_group_name --location $location 

3. Create a New Blob Container:

In [ ]:
container_name="scripts"
!az storage container create -n $container_name --account-name $storage_account_name --fail-on-exist

4. Upload the Demo Application to our Blob Storage:

In [ ]:
file_name="demo-app-bundle.tar.gz"
file_path="./bundle/demo-app-bundle.tar.gz" 
!az storage blob upload --container-name $container_name --file $file_path --name $file_name --account-name $storage_account_name

5. Deploy our ARM Template:
   - The ARM Template will create a Linux VMSS with a custom script extension that will download our a demo application named `demo-app-bundle.tar.gz` from the Blob Storage from Blob Storage, extract and run the demo application.

In [ ]:
vmss_name="" #Add a name to the VMSS
admin_username="" #Add an Username for VMSS VMs
admin_password="" #Add an Password for VMSS VMs
template_file_path="./deployment/Templates/ARM/Linux/azuredeploy.json"
!az deployment group create --resource-group $resource_group_name --template-file $template_file_path --parameters vmssName=$vmss_name adminUsername=$admin_username adminPassword=$admin_password storageAccountName=$storage_account_name